# Preparação da Rede Neural

- A arquitetura do modelo foi alocada em uma documentação a parte, ela pode ser vista a partir do seguinte link: https://github.com/Inteli-College/2024-2A-T04-SI11-G01/blob/main/documents/Sprint%202/Programa%C3%A7%C3%A3o%20-%20Sprint%202%20-%20Arquitetura%20da%20rede.md

# Separação treino e teste

In [ ]:
df_pipe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5397 entries, 17857536 to 17837502
Data columns (total 68 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   MATRICULA                                 5397 non-null   int64         
 1   EMP_CODIGO                                5397 non-null   int64         
 2   COD_GRUPO                                 5397 non-null   int64         
 3   COD_SETOR_COMERCIAL                       5397 non-null   int64         
 4   NUM_QUADRA                                5397 non-null   int64         
 5   COD_ROTA_LEITURA                          5397 non-null   int64         
 6   SEQ_RESPONSAVEL                           5397 non-null   int64         
 7   ECO_RESIDENCIAL                           5397 non-null   int64         
 8   ECO_COMERCIAL                             5397 non-null   int64         
 9   ECO_INDUSTRIAL          

In [ ]:
# Separando as features e o alvo
take_off_columns = ['fraude', 'DAT_LEITURA', 'STA_ACEITA_LEITURA', 'EXCECAO', 'STA_TROCA', 'FATURADO_MEDIA', 'HORA_LEITURA', 'DSC_OCORRENCIA', 'SUB_CATEGORIA', 'CATEGORIA', 'TIPO_LIGACAO']
X = df_pipe.drop(take_off_columns, axis=1)
y = df_pipe['fraude']


- reanliasar essa parte devido o downsamplin

In [ ]:
# Analisando o desequilíbrio das classes
y.value_counts().plot(kind='bar')
plt.title('Distribuição das Classes')
plt.show()

# Aplicando SMOTE para balanceamento
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Verificando a nova distribuição das classes
y_balanced.value_counts().plot(kind='bar')
plt.title('Distribuição das Classes Após SMOTE')
plt.show()

In [ ]:
# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_balanced, y_balanced, test_size=0.3,  random_state=42)

# Compilação do modelo e estruturação da rede

In [ ]:
def create_model():
  # Define a arquitetura do modelo (arquitetura em pirâmide)
  return Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
  ])

In [ ]:
model = create_model()

# Compilando o modelo
model.compile(
      optimizer= Lion(learning_rate=0.01),
      loss='binary_crossentropy',
      metrics=['accuracy',
               tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall()])

model.summary()

In [ ]:
# Definindo o early stopping
earlyStop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True) # aqui demonstra a utilização do early stopping, tomar cuidado pra quando iremos parar

# Aplicação do dataset na rede e Treinamento

- testar rodar o modelo, tirando o earlystopping

In [ ]:
# Treinando o modelo
history = model.fit(X_train,
                    y_train,
                    epochs=120,
                    batch_size=64,
                    callbacks=[earlyStop],
                    validation_data=(X_test, y_test))

Epoch 1/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5402 - loss: 129781.8438 - val_accuracy: 0.5639 - val_loss: 0.6854
Epoch 2/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5567 - loss: 0.6875 - val_accuracy: 0.5639 - val_loss: 0.6880
Epoch 3/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5602 - loss: 0.6880 - val_accuracy: 0.5639 - val_loss: 0.6850
Epoch 4/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5450 - loss: 0.6910 - val_accuracy: 0.5639 - val_loss: 0.6850
Epoch 5/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5585 - loss: 0.6877 - val_accuracy: 0.5639 - val_loss: 0.6850
Epoch 6/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5602 - loss: 0.6870 - val_accuracy: 0.5639 - val_loss: 0.6910
Epoch 7/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5462 - loss: 0.6898 - val_accuracy: 0.5639 - val_loss: 0.6851
Epoch 8/40
432/432 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5702 - loss: 0.6840 - val_accurac

In [ ]:
# Avaliar o modelo no conjunto de teste
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}, F1-Score: {f1_score}, Precision: {precision}, Recall: {recall}')

# Analise de resultados

In [ ]:
# Preparando os dados para o plot
epochs = list(range(1, len(history.history['loss']) + 1))
history_df = pd.DataFrame({
    'Epoch': epochs,
    'Training Loss': history.history['loss'],
    'Validation Loss': history.history['val_loss'],
    'Training Accuracy': history.history['accuracy'],
    'Validation Accuracy': history.history['val_accuracy']
})

# Plotando a perda durante o treinamento
fig_loss = px.line(history_df, x='Epoch', y=['Training Loss', 'Validation Loss'],
                   labels={'value': 'Loss', 'variable': 'Type'},
                   title='Loss during Training')
fig_loss.show()

# Plotando a acurácia durante o treinamento
fig_accuracy = px.line(history_df, x='Epoch', y=['Training Accuracy', 'Validation Accuracy'],
                       labels={'value': 'Accuracy', 'variable': 'Type'},
                       title='Accuracy during Training')
fig_accuracy.show()

# Import do modelo para arquivo pickel - .pkl